<a href="https://colab.research.google.com/github/jakeengstrom3/SmartRecyclingBin_Capstone/blob/main/Image_Classification_Transfer_Learning_from_Images_on_Local_Disk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
#!ls /mydrive

# Image Classification Transfer learning example with dataset from local disk




This Colab follows closely the https://www.tensorflow.org/lite/models/modify/model_maker/image_classification example, but with some small changes to upload custom images instead of downloading the example dataset

# Step 0: Prerequisites

In [ ]:
import os

In [ ]:
!pip install -q tflite-model-maker

In [ ]:
import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

# Step 1: Loading data from local disk into TF ImageDataset object
**Before running, go to the folder icon and drag a zipped dataset into /content/.**

The dataset must have images properly sorted into labeled sub directories. Each sub directory will correspond to a new class in the models head layer. For example, make sure all images of plastic bottles are in the directory dataset/plastic_bottles/. 

In [ ]:
!cd  /mydrive/
#!unzip -q /mydrive/Recyclables.zip
#!unzip -q /mydrive/rawimgsEdited2.zip
#!unzip -q /mydrive/Recyclables_updated.zip
#!unzip -q /mydrive/Recyclables-update+rotate.zip
!unzip -q /mydrive/recycle_images_psu.zip

Set image path to the name of the uploaded data folder

In [ ]:
#image_path ='Recyclables'
#image_path ='rawimgsEdited2'
#image_path = 'Recyclables_updated'
#image_path ='Recyclables-update+rotate'
image_path ='recycle_images_psu'


Using tflife_model_maker/image_classifer/Dataloader to load the data from folder. This will automatically label the images with the name of thier resident directory. 

In [ ]:
data = DataLoader.from_folder(image_path)

print(data.size)

Showing an example of 25 images to make sure the data was loaded in properly

In [ ]:
plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(data.gen_dataset().unbatch().take(25)):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)
  plt.xlabel(data.index_to_label[label.numpy()])
plt.show()

## Train test splitting
Here we are splitting the data into training, validation, and test data, with a 0.8 to 0.1 to 0.1 split, respectively. 

In [ ]:
train_data, rest_data = data.split(0.9)
validation_data, test_data = rest_data.split(0.5)

In [ ]:
print(type(train_data))

# Step 2: Customizing the TF model

Choose pretrained model to customize. Options using ModelMaker are: 
* 'efficientnet_lite0',
*'efficientnet_lite1',
*'efficientnet_lite2',
*'efficientnet_lite3',
*'efficientnet_lite4',
*'mobilenet_v2',
*'resnet_50'

In [ ]:
model_spec='mobilenet_v2'

In [ ]:
model = image_classifier.create(
  train_data, 
  validation_data=validation_data,
  model_spec=model_spec,
  batch_size = 256,
  epochs= 15

)

# Step 3: Evalutate the model

In [ ]:
loss, accuracy = model.evaluate(test_data)


In [ ]:
y_test = []
y_pred = []

In [ ]:
y_test = []
y_pred = []
# A helper function that returns 'red'/'black' depending on if its two input
# parameter matches or not.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

# Then plot 100 test images and their predicted labels.
# If a prediction result is different from the label provided label in "test"
# dataset, we will highlight it in red color.
plt.figure(figsize=(20, 20))
predicts = model.predict_top_k(test_data)
missed = 0
#for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(100)):
for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(len(test_data))):

  predict_label = predicts[i][0][0]
  color = get_label_color(predict_label,
                          test_data.index_to_label[label.numpy()])
  y_pred.append(predict_label)
  y_test.append(test_data.index_to_label[label.numpy()])
  if color == 'red':
    missed+=1
    plt.figure(figsize=(4,2), dpi=60)
    plt.imshow(image.numpy(), cmap=plt.cm.gray)
    plt.xlabel('Predicted: %s' % predict_label + 
              '\nActual: %s' % test_data.index_to_label[label.numpy()] 
               , color= 'red')
    
plt.show()
print("missed total = ", missed)
print("out of = ", len(y_test) )


# Export the model

In [ ]:
model.export(export_dir='.', tflite_filename= 'mobilenet_v2_psu_data.tflite')

In [ ]:
# Download the TFLite model to your local computer.
from google.colab import files
files.download('mobilenet_v2_psu_data.tflite')

In [ ]:
"""x = [i[0] for i in predicts]
final_labels = [i[0] for i in x]"""


#Metrics

In [ ]:
#model.evaluate(validation_data)
#model.evaluate(test_data)

In [ ]:

from sklearn import metrics
from sklearn.metrics import confusion_matrix #
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support #
from sklearn.metrics import classification_report #
from sklearn.metrics import ConfusionMatrixDisplay #

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
cm_display = ConfusionMatrixDisplay(confusion_matrix = cm, 
                                    #display_labels = ["can","glass","paperB","plastic","NRplasticB","NRbottle","NRcup","NRrecipt"])
                                    display_labels = ["Aluminum","Glass","Plastic", "NR", "smth", "smth2"])


cm_display.plot()
plt.title("mobilenet (epoch:5, batch size:256)")
plt.show()

In [ ]:
print(precision_recall_fscore_support(y_test, y_pred, average='weighted'))

In [ ]:
#diff accuracy than tf acuuracy
#beacuse keras and sklearn seem to calculate
#multiclass accuracy differntly
print(accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
10print( len(y_pred) )
print( len(rest_data) )
print( len(train_data) )
print("total missed= ", missed)
print("% missed= ", 100 * (missed/len(y_pred)) )

# Local Test



In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      
      video.videoHeight = 224 //300
      video.videoWidth = 224 //300
      video.height = 224 //300
      video.width = 224 //300
      
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = 224 //video.videoWidth; //300
      canvas.height = 224 //video.videoHeight; //300
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
#print(filename.shape)
#print(Image(filename))
#filename = (np.expand_dims(filename,0))
#print(filename.shape)
from keras.utils import np_utils
#filename=np_utils.to_categorical(filename)


#image = img_to_array(filename)
#image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))


# Import required libraries
import tensorflow as tf
from PIL import Image
# Read a PIL image  
img = Image.open(filename)#('image.jpg')
print(img)
#img = tf.image.resize(img, [300,300]).shape.as_list()

# Convert the PIL image to Tensor
img_to_tensor = tf.convert_to_tensor(img)
# print the converted Torch tensor
#print(img_to_tensor)
#print("dtype of tensor:",img_to_tensor.dtype)
#print(img_to_tensor.shape)


In [ ]:
#############################################
filename = (np.expand_dims(filename,0))
#print(filename.shape)
#kk=tf.reshape(img_to_tensor, [300, 300, 3])
#print(kk.shape)
j = (np.expand_dims(img_to_tensor,0))
predicts = model.predict_top_k(j)
predict_label = predicts[0][0][0]

print(predict_label)
print(j.shape)

In [ ]:
print(test_data)
print(type(test_data))
print(filename)

#Test diff epochs

In [ ]:
for i in range(6):
  model = image_classifier.create(
    train_data, 
    validation_data=validation_data,
    model_spec=model_spec,
    epochs= i * 5
  )

  loss, accuracy = model.evaluate(test_data)